# prep the model and tokeniser + test on one sentence first

In [16]:
from torch import nn

class DistilBERTClassifier(nn.Module):
    def __init__(self, num_labels=3):
        super(DistilBERTClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.pre_classifier = nn.Linear(768, 768)
        self.classifier = nn.Linear(768, num_labels)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        output_1 = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooled_output = hidden_state[:, 0]
        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = nn.ReLU()(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [17]:
model.eval()

DistilBERTClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

In [18]:
import torch 

model = torch.load('/Users/sriyan/Documents/techjam/final-sentiment-analysis/distilBERT/models/full_model.pt')


/var/folders/wk/27zdp1q12l9dx34w5ml48nrw0000gn/T/ipykernel_58937/3163128231.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/Users/sriyan/Documents/t

In [19]:
# test out with one sentence

import torch
import torch.nn.functional as F

loaded_tokenizer =DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

def predict_sentiment(model, tokenizer, text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = F.softmax(outputs, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1)
    sentiment_labels = ["Negative", "Neutral", "Positive"]
    predicted_sentiment = sentiment_labels[predicted_class.item()]

    return predicted_sentiment, probabilities.squeeze().tolist()

text = "Can DistilBERT work with labella? We will find out tonight on CBN Live Tonight at 7pm EST, stay tuned!"
sentiment, probabilities = predict_sentiment(model, loaded_tokenizer, text)

print(f"Input text: {text}")
print(f"Predicted sentiment: {sentiment}")
print(f"Probabilities: Negative: {probabilities[0]:.4f}, Neutral: {probabilities[1]:.4f}, Positive: {probabilities[2]:.4f}")

Input text: Can DistilBERT work with labella? We will find out tonight on CBN Live Tonight at 7pm EST, stay tuned!
Predicted sentiment: Neutral
Probabilities: Negative: 0.0023, Neutral: 0.9905, Positive: 0.0073


# using the fine tuned model for testing purposes

In [21]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel

In [22]:
test_df = pd.read_csv('/Users/sriyan/Documents/techjam/final-sentiment-analysis/distilBERT/data/twitter_validation.csv', header=None)

In [23]:
test_df.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [12]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBERTClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

In [24]:
test_df.rename(columns={0: 'tweet_id', 1: 'topic', 2:'true_sentiment', 3:'text'}, inplace=True)
test_df.head()

,tweet_id,topic,true_sentiment,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [25]:
test_df=test_df[test_df["true_sentiment"] != "Irrelevant"]
test_df.head()


,tweet_id,topic,true_sentiment,text
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
5,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [26]:
sentiments = []
probabilities_list = []

for text in test_df['text']:
    sentiment, probabilities = predict_sentiment(model, loaded_tokenizer, text)
    sentiments.append(sentiment)
    probabilities_list.append(probabilities)

In [27]:
test_df['predicted_sentiment'] = sentiments
test_df['probabilities'] = probabilities_list

In [28]:
test_df.to_csv('prediction.csv', index=False)